# LPA x Dataset Crosstab

This notebook produces a crosstab showing LPAs and the count of entitiyesd in each dataset

|entity|dataset1|dataset2|
|---|---|---|
|Chigley Borough Council| 2| 1|
|Fulchester County| 1| 0|

It needs a recentyly downloaded copy of the entity datasette from https://datasette.planning.data.gov.uk/entity which it uses as input. You can see the input and output file names in the next section of this notebook.

In [1]:
source_file = "/mnt/c/Users/MarkSmith/Downloads/entity_2023_09_12.sqlite3" # or whatever you called your download.
dest_file=  "/mnt/c/Users/MarkSmith/Downloads/entity_2023_09_12_crosstab.csv" # or wherever you want your output

First time to run is a bit slow as it indexes the entity table.

In [2]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(source_file)

cursor = cnx.cursor()

cnx.execute("CREATE INDEX if not exists idx_entity ON entity(dataset)") 
cnx.execute("CREATE INDEX if not exists idx_organisation_entity ON entity(organisation_entity)") 


In [3]:
lpas = pd.read_sql_query("SELECT entity, naME, REFERENCE FROM Entity where dataset = 'local-authority'", cnx)
lpas['entity'] = lpas['entity'].astype(str) # there's a type error when you concet later if you don't do this.


lpas

entity                            name   reference
0         26           Adur District Council         ADU
1         27       Allerdale Borough Council         ALL
2         28    Amber Valley Borough Council         AMB
3         29           Arun District Council         ARU
4         30         Ashford Borough Council         ASF
..       ...                             ...         ...
371   501909    West Northamtonshire Council        WNUA
372  8700000             Avon County Council    Q4829191
373  8700001       Wansbeck District Council  Q105544736
374  8700002  North Northamptonshire Council  Q111232446
375  8700003   West Northamptonshire Council  Q111232469

[376 rows x 3 columns]

In [4]:
grouped = pd.read_sql_query(F"SELECT organisation_entity as entity, dataset, count(entity) as count FROM Entity  group by 1,2", cnx)

grouped

entity                           dataset  count
0                             brownfield-land    650
1                             brownfield-site      9
2                     central-activities-zone      8
3                               document-type      7
4                    local-planning-authority      3
...     ...                               ...    ...
1048     97  infrastructure-funding-statement      1
1049     98                   brownfield-land     43
1050     98  infrastructure-funding-statement      1
1051     99                   brownfield-land     35
1052     99  infrastructure-funding-statement      1

[1053 rows x 3 columns]

In [5]:
pivot = grouped.pivot(index = 'entity', columns='dataset', values='count')

pivot

dataset  ancient-woodland  ancient-woodland-status  \
entity                                               
                      NaN                      NaN   
1                     NaN                      NaN   
10                    NaN                      NaN   
100                   NaN                      NaN   
101                   NaN                      NaN   
...                   ...                      ...   
95                    NaN                      NaN   
96                    NaN                      NaN   
97                    NaN                      NaN   
98                    NaN                      NaN   
99                    NaN                      NaN   

dataset  area-of-outstanding-natural-beauty  article-4-direction  \
entity                                                             
                                        NaN                  NaN   
1                                       NaN                  NaN   
10                                      NaN                  NaN   
100                                     NaN                  NaN   
101                                     NaN                  NaN   
...                                     ...                  ...   
95                                      NaN                  NaN   
96                                      NaN                  NaN   
97                                      NaN                  NaN   
98                                      NaN                  NaN   
99                                      NaN                  NaN   

dataset  article-4-direction-area  article-4-direction-rule  battlefield  \
entity                                                                     
                              NaN                       NaN          NaN   
1                             NaN                       NaN          NaN   
10                            NaN                       NaN          NaN   
100                           NaN                       NaN          NaN   
101                           NaN                       NaN          NaN   
...                           ...                       ...          ...   
95                            NaN                       NaN          NaN   
96                            NaN                       NaN          NaN   
97                            NaN                       NaN          NaN   
98                            NaN                       NaN          NaN   
99                            NaN                       NaN          NaN   

dataset  brownfield-land  brownfield-site  building-preservation-notice  ...  \
entity                                                                   ...   
                   650.0              9.0                           NaN  ...   
1                   79.0             41.0                           NaN  ...   
10                   NaN              NaN                           NaN  ...   
100                545.0              NaN                           NaN  ...   
101                 40.0              NaN                           NaN  ...   
...                  ...              ...                           ...  ...   
95                  33.0              NaN                           NaN  ...   
96                 126.0              NaN                           NaN  ...   
97                   NaN              NaN                           NaN  ...   
98                  43.0              NaN                           NaN  ...   
99                  35.0              NaN                           NaN  ...   

dataset  site-category  site-of-special-scientific-interest  \
entity                                                        
                   2.0                                  NaN   
1                  NaN                                  NaN   
10                 NaN                                  NaN   
100                NaN                                  NaN   
101 

In [6]:
result = pd.merge(lpas, pivot, on= 'entity',  how = "inner")

result.to_csv(dest_file)

result

entity                          name reference  ancient-woodland  \
0       26         Adur District Council       ADU               NaN   
1       27     Allerdale Borough Council       ALL               NaN   
2       28  Amber Valley Borough Council       AMB               NaN   
3       29         Arun District Council       ARU               NaN   
4       30       Ashford Borough Council       ASF               NaN   
..     ...                           ...       ...               ...   
343    390     Wychavon District Council       WYC               NaN   
344    392  Wyre Forest District Council       WYE               NaN   
345    393      Wycombe District Council       WYO               NaN   
346    394          Wyre Borough Council       WYR               NaN   
347    395          City of York Council       YOR               NaN   

     ancient-woodland-status  area-of-outstanding-natural-beauty  \
0                        NaN                                 NaN   
1                        NaN                                 NaN   
2                        NaN                                 NaN   
3                        NaN                                 NaN   
4                        NaN                                 NaN   
..                       ...                                 ...   
343                      NaN                                 NaN   
344                      NaN                                 NaN   
345                      NaN                                 NaN   
346                      NaN                                 NaN   
347                      NaN                                 NaN   

     article-4-direction  article-4-direction-area  article-4-direction-rule  \
0                    NaN                       NaN                       NaN   
1                    NaN                       NaN                       NaN   
2                    NaN                       NaN                       NaN   
3                    NaN                       NaN                       NaN   
4                    NaN                       NaN                       NaN   
..                   ...                       ...                       ...   
343                  NaN                       NaN                       NaN   
344                  NaN                       NaN                       NaN   
345                  NaN                       NaN                       NaN   
346                  NaN                       NaN                       NaN   
347                  NaN                       NaN                       NaN   

     battlefield  ...  site-category  site-of-special-scientific-interest  \
0            NaN  ...            NaN                                  NaN   
1            NaN  ...            NaN                                  NaN   
2            NaN  ...            NaN                                  NaN   
3            NaN  ...            NaN                                  NaN   
4            NaN  ...            NaN                                  NaN   
..           ...  ...            ...                                  ...   
343          NaN  ...            NaN                                  NaN   
344          NaN  ...            NaN                                  NaN   
345          NaN  ...            NaN                                  NaN   
346          NaN  ...            NaN                                  NaN   
347          NaN  ...            NaN                                  NaN   

     special-area-of-conservation  special-protection-area  tree  \
0                             NaN                      NaN   NaN   
1                             NaN                      NaN   NaN   
2                             NaN                      NaN   NaN   
3                             NaN                      NaN   NaN   
4                             NaN                      NaN   NaN   
..                          